In [89]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.svm import LinearSVC
from sklearn import cross_validation as cv
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,accuracy_score
from scipy.sparse import csr_matrix,hstack
import time
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, SelectKBest
from sklearn.cross_validation import StratifiedKFold,KFold
seed = 7
np.random.seed(seed)

In [2]:
events = pd.read_csv('input/events.csv',index_col='event_id')
label = pd.read_csv('input/label_categories.csv')
app_event = pd.read_csv('input/app_events.csv')
app_label = pd.read_csv('input/app_labels.csv')
device = pd.read_csv('input/phone_brand_device_model.csv')
device = device.drop_duplicates('device_id').set_index('device_id')
gender_age_train = pd.read_csv('input/gender_age_train.csv',index_col = 'device_id')
gender_age_test = pd.read_csv('input/gender_age_test.csv',index_col = 'device_id')

For the convinience of creating sparse matrix, we create another integer index for the training and testing set.

In [3]:
gender_age_train['int_index'] = np.arange(gender_age_train.shape[0])
gender_age_test['int_index'] = np.arange(gender_age_test.shape[0])

In [4]:
print('gender_age_train\n',gender_age_train.head(1))
print('\ngender_age_test\n',gender_age_test.head(1))
print('\ndevice\n',device.head(1))
print('\nevents\n',events.head(1))
print('\nlabel\n',label.head(1))
print('\napp_event\n',app_event.head(1))
print('\napp_label\n',app_label.head(1))

gender_age_train
                      gender  age   group  int_index
device_id                                          
-8076087639492063270      M   35  M32-38          0

gender_age_test
                      int_index
device_id                     
1002079943728939269          0

device
                      phone_brand device_model
device_id                                    
-8890648629457979026          小米           红米

events
                   device_id            timestamp  longitude  latitude
event_id                                                             
1         29182687948017175  2016-05-01 00:55:25     121.38     31.24

label
    label_id category
0         1      NaN

app_event
    event_id               app_id  is_installed  is_active
0         2  5927333115845830913             1          1

app_label
                 app_id  label_id
0  7324884708820027918       251


# Feature engineering I: phone brand

Encoding the phone brand, and feed this as a feature into the training and testing test.

In [5]:
encoder = LabelEncoder()
encoder.fit(device.phone_brand)
device['brand'] = encoder.transform(device.phone_brand)
gender_age_train['brand'] = device['brand']
gender_age_test['brand'] = device['brand']

Create sparse matrix with each row representing one device and each column representing one brand

In [6]:
X_train_brand = csr_matrix((np.ones(gender_age_train.shape[0]),
                            (gender_age_train.int_index,gender_age_train.brand)))
X_test_brand = csr_matrix((np.ones(gender_age_test.shape[0]),
                           (gender_age_test.int_index,gender_age_test.brand)))
print('X_train_brand shape:',X_train_brand.shape)
print('X_test_brand shape:',X_test_brand.shape)

X_train_brand shape: (74645, 131)
X_test_brand shape: (112071, 131)


# Feature engineering II: Installed app

Encoding the app_id and store it into app column, and feed this as a feature into the training and testing test.

In [7]:
encoder2 = LabelEncoder()
encoder2.fit(app_event.app_id)
app_event['app'] = encoder2.transform(app_event.app_id)

In [8]:
np.sort(app_event.app.unique())

array([    0,     1,     2, ..., 19234, 19235, 19236])

In [9]:
print(app_event.head(1))
print(events.head(1))

   event_id               app_id  is_installed  is_active    app
0         2  5927333115845830913             1          1  15408
                  device_id            timestamp  longitude  latitude
event_id                                                             
1         29182687948017175  2016-05-01 00:55:25     121.38     31.24


In [10]:
installed_app = pd.merge(events[['device_id']],app_event[['event_id','app','is_installed']],
                         how='right',right_on = 'event_id',left_index = True)
print(installed_app.head())

             device_id  event_id    app  is_installed
0 -6401643145415154744         2  15408             1
1 -6401643145415154744         2   3384             1
2 -6401643145415154744         2   7620             1
3 -6401643145415154744         2   8902             1
4 -6401643145415154744         2  18686             1


In [11]:
installed_app_grouped = installed_app.groupby(['device_id','app'])['app'].agg(['size'])
print('installed_app_grouped:')
print(installed_app_grouped.head())
installed_app_train = pd.merge(installed_app_grouped,gender_age_train[['int_index']],
                               how = 'right',right_index=True,left_index=True)
installed_app_test = pd.merge(installed_app_grouped,gender_age_test[['int_index']],
                              how = 'right',right_index=True,left_index=True)
print('installed_app_train:')
print(installed_app_train.head())

installed_app_grouped:
                           size
device_id            app       
-9222956879900151005 548     18
                     1096    18
                     1248    26
                     1545    12
                     1664    18
installed_app_train:
                           size  int_index
device_id            app                  
-9222956879900151005 548     18      21594
                     1096    18      21594
                     1248    26      21594
                     1545    12      21594
                     1664    18      21594


In [12]:
installed_app_train = installed_app_train.reset_index()
installed_app_test = installed_app_test.reset_index()
installed_app_train = installed_app_train.dropna(subset=['int_index'])
installed_app_test = installed_app_test.dropna(subset=['int_index'])
print(installed_app_train.head())
print(installed_app_test.head())

             device_id   app  size  int_index
0 -9222956879900151005   548    18      21594
1 -9222956879900151005  1096    18      21594
2 -9222956879900151005  1248    26      21594
3 -9222956879900151005  1545    12      21594
4 -9222956879900151005  1664    18      21594
             device_id    app  size  int_index
0 -9222661944218806987   1867     3      13612
1 -9222661944218806987   7519     8      13612
2 -9222661944218806987   7843     1      13612
3 -9222661944218806987   8704     4      13612
4 -9222661944218806987  10000     1      13612


Calculate the number of unique app:

In [13]:
print(gender_age_test.shape[0])
print(installed_app_train.shape[0])

112071
915632


In [14]:
print('The number of unique apps:')
print(np.size(installed_app.app.unique()))
appnumber = np.size(installed_app.app.unique())

The number of unique apps:
19237


In [15]:
print(np.sort(installed_app_train.app.unique()))
print(installed_app_test.shape[0])

[    0     1     2 ..., 19234 19235 19236]
1387337


In [16]:
X_train_installed = csr_matrix((np.ones(installed_app_train.shape[0]),
                                (installed_app_train.int_index,installed_app_train.app)), 
                               shape = (gender_age_train.shape[0],appnumber))
X_test_installed = csr_matrix((np.ones(installed_app_test.shape[0]),
                               (installed_app_test.int_index,installed_app_test.app)),
                               shape = (gender_age_test.shape[0],appnumber))
print('X_train_installed shape:',X_train_installed.shape)
print('X_test_installed shape:',X_test_installed.shape)

X_train_installed shape: (74645, 19237)
X_test_installed shape: (112071, 19237)


In [17]:
np.sort(installed_app_test.int_index.unique())

array([     0,      1,      2, ..., 112059, 112060, 112063])

# Feature engineering III: phone device model

In [18]:
brand_model = device.phone_brand.str.cat(device.device_model)
encoder3 = LabelEncoder()
encoder3.fit(brand_model)
device['model'] = encoder3.transform(brand_model)
gender_age_train['model'] = device['model']
gender_age_test['model'] = device['model']

In [19]:
X_train_model = csr_matrix((np.ones(gender_age_train.shape[0]),(gender_age_train.int_index,gender_age_train.model)))
X_test_model = csr_matrix((np.ones(gender_age_test.shape[0]),(gender_age_test.int_index,gender_age_test.model)))
print('X_train_brand shape:',X_train_model.shape)
print('X_test_brand shape:',X_test_model.shape)

X_train_brand shape: (74645, 1667)
X_test_brand shape: (112071, 1667)


# Feature engineering IV: app label

In [20]:
print(app_event[['app_id','event_id']].head(1))
print(app_label[['app_id','label_id']].head(1))

                app_id  event_id
0  5927333115845830913         2
                app_id  label_id
0  7324884708820027918       251


In [21]:
app_label_new = app_label.loc[app_label.app_id.isin(app_event.app_id.unique())].copy()
app_label_new['app'] = encoder2.transform(app_label_new.app_id)
encoder4 = LabelEncoder().fit(app_label_new.label_id)
app_label_new['label'] = encoder4.transform(app_label_new.label_id)
labelnumber = len(encoder4.classes_)
print('app_label_new:')
print(app_label_new.head())

app_label_new:
                app_id  label_id    app  label
0  7324884708820027918       251  17355    207
1 -4494216993218550286       251   4618    207
2  6058196446775239644       406  15548    247
3  6058196446775239644       407  15548    248
4  8694625920731541625       406  18689    247


In [22]:
print(app_label.size)
print(installed_app.size)
print('installed_app_grouped:')
print(installed_app_grouped.head())
installed_label_grouped = (installed_app_grouped.reset_index()[['device_id','app']]
                          .merge(app_label_new[['app','label']])
                          .groupby(['device_id','label']))['app'].agg(['size']).reset_index()
                          
print('installed_label_grouped:')
print(installed_label_grouped.head())

919886
129892268
installed_app_grouped:
                           size
device_id            app       
-9222956879900151005 548     18
                     1096    18
                     1248    26
                     1545    12
                     1664    18
installed_label_grouped:
             device_id  label  size
0 -9222956879900151005    117     1
1 -9222956879900151005    120     1
2 -9222956879900151005    126     1
3 -9222956879900151005    138     2
4 -9222956879900151005    147     2


In [23]:
label_app_train = pd.merge(installed_label_grouped,gender_age_train[['int_index']],
                               how = 'right',right_index=True,left_on='device_id')
label_app_test = pd.merge(installed_label_grouped,gender_age_test[['int_index']],
                              how = 'right',right_index=True,left_on ='device_id' )
label_app_train = label_app_train.dropna(subset= ['int_index','label'])
label_app_test = label_app_test.dropna(subset= ['int_index','label'])

In [24]:
X_train_label = csr_matrix((np.ones(label_app_train.shape[0]),(label_app_train.int_index,label_app_train.label)),
                              shape = (gender_age_train.shape[0],labelnumber))
X_test_label = csr_matrix((np.ones(label_app_test.shape[0]),(label_app_test.int_index,label_app_test.label)),
                              shape = (gender_age_test.shape[0],labelnumber))
print('X_train_installed shape:',X_train_label.shape)
print('X_test_installed shape:',X_test_label.shape)

X_train_installed shape: (74645, 492)
X_test_installed shape: (112071, 492)


# Feature engineering V: active app

In [25]:
active_app = pd.merge(events[['device_id']],app_event[['event_id','app','is_active']],
                         how='right',right_on = 'event_id',left_index = True)
active_app = active_app.loc[active_app.is_active==1]
print(active_app.head())

             device_id  event_id    app  is_active
0 -6401643145415154744         2  15408          1
3 -6401643145415154744         2   8902          1
4 -6401643145415154744         2  18686          1
5 -6401643145415154744         2  14346          1
9 -6401643145415154744         2  16908          1


In [26]:
active_app_grouped = active_app.groupby(['device_id','app'])['app'].agg(['size'])
print('active_app_grouped:')
print(active_app_grouped.head())

active_app_grouped:
                           size
device_id            app       
-9222956879900151005 548      4
                     1248    15
                     1545     2
                     1848    31
                     2236    17


In [27]:
active_app_train = pd.merge(active_app_grouped,gender_age_train[['int_index']],
                               how = 'left',right_index=True,left_index=True)
active_app_test = pd.merge(active_app_grouped,gender_age_test[['int_index']],
                              how = 'left',right_index=True,left_index=True)
print('active_app_train:')
print(active_app_train.head())

active_app_train:
                           size  int_index
device_id            app                  
-9222956879900151005 548      4    21594.0
                     1248    15    21594.0
                     1545     2    21594.0
                     1848    31    21594.0
                     2236    17    21594.0


In [28]:
active_app_train = active_app_train.reset_index()
active_app_test = active_app_test.reset_index()
active_app_train = active_app_train.dropna(subset=['int_index'])
active_app_test = active_app_test.dropna(subset=['int_index'])
print(active_app_train.head())
print(active_app_test.head())

             device_id   app  size  int_index
0 -9222956879900151005   548     4    21594.0
1 -9222956879900151005  1248    15    21594.0
2 -9222956879900151005  1545     2    21594.0
3 -9222956879900151005  1848    31    21594.0
4 -9222956879900151005  2236    17    21594.0
              device_id    app  size  int_index
55 -9222661944218806987   1867     3    13612.0
56 -9222661944218806987   7519     7    13612.0
57 -9222661944218806987   7843     1    13612.0
58 -9222661944218806987   8704     3    13612.0
59 -9222661944218806987  10000     1    13612.0


In [29]:
print(np.sort(active_app_train.app.unique()))
print(appnumber)

[    0     5     6 ..., 19225 19228 19236]
19237


In [30]:
appnumber = np.size(installed_app.app.unique())
# binary active map
#X_train_active = csr_matrix((np.ones(active_app_train.shape[0]),
#                            (active_app_train.int_index,active_app_train.app)), 
#                            shape = (gender_age_train.shape[0],appnumber))
#X_test_active = csr_matrix((np.ones(active_app_test.shape[0]),
#                            (active_app_test.int_index,active_app_test.app)),
#                            shape = (gender_age_test.shape[0],appnumber))
# count the number of active app
X_train_active = csr_matrix((np.log(np.log(active_app_train['size']+1)+1),
                            (active_app_train.int_index,active_app_train.app)), 
                            shape = (gender_age_train.shape[0],appnumber))
X_test_active = csr_matrix((np.log(np.log(active_app_test['size']+1)+1),
                            (active_app_test.int_index,active_app_test.app)),
                            shape = (gender_age_test.shape[0],appnumber))
print('X_train_active shape:',X_train_active.shape)
print('X_test_active shape:',X_test_active.shape)

X_train_active shape: (74645, 19237)
X_test_active shape: (112071, 19237)


# Feature engineering VI: active time period

In [31]:
events.head()

,device_id,timestamp,longitude,latitude
event_id,,,,
1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


In [32]:
events_time = events[['device_id','timestamp']].copy()
events_time['time']  = events_time['timestamp'].str[11:13].astype(int)
events_time.drop(['timestamp'],axis=1,inplace=True)
events_time = events_time.groupby(['device_id','time'])['time'].agg({'times':'count'}).reset_index()
print(events_time.head())
timenumber= events_time.time.unique().shape[0]

             device_id  time  times
0 -9222956879900151005     7      2
1 -9222956879900151005    11      7
2 -9222956879900151005    12     13
3 -9222956879900151005    13      3
4 -9222956879900151005    14      5


In [33]:
time_train = pd.merge(events_time,gender_age_train[['int_index']],
                               right_index=True,left_on='device_id')
time_test = pd.merge(events_time,gender_age_test[['int_index']],
                               right_index=True,left_on ='device_id' )
#binary
#X_train_time = csr_matrix((np.ones(time_train.shape[0]),
#                            (time_train.int_index,time_train.time)), 
#                            shape = (gender_age_train.shape[0],timenumber))
#X_test_time = csr_matrix((np.ones(time_test.shape[0]),
#                            (time_test.int_index,time_test.time)),
#                            shape = (gender_age_test.shape[0],timenumber))

#number
X_train_time = csr_matrix((np.log(np.log(time_train['times']+1)+1),
                            (time_train.int_index,time_train.time)), 
                            shape = (gender_age_train.shape[0],timenumber))
X_test_time = csr_matrix((np.log(np.log(time_test['times']+1)+1),
                            (time_test.int_index,time_test.time)),
                            shape = (gender_age_test.shape[0],timenumber))
print('X_train_time shape:',X_train_time.shape)
print('X_test_time shape:',X_test_time.shape)

X_train_time shape: (74645, 24)
X_test_time shape: (112071, 24)


Normalization of the time period count.

In [34]:
scaler = StandardScaler(with_mean=False)
X_train_time = scaler.fit_transform(X_train_time)
X_test_time = scaler.transform(X_test_time)

# feature join and selection

In [105]:
X_train_total = hstack((X_train_brand,X_train_model,X_train_installed,X_train_label),format='csr')
X_test_total = hstack((X_test_brand,X_test_model,X_test_installed,X_test_label),format='csr')
print('Training shape:')
print(X_train_total.shape)
print('Testing shape:')
print(X_test_total.shape)

Training shape:
(74645, 21527)
Testing shape:
(112071, 21527)


In [106]:
#percentile selction
#selector = SelectPercentile(f_classif, percentile=80)
#selector.fit(X_train_total, y_train_total)
#X_train_total = selector.transform(X_train_total)
#X_test_total = selector.transform(X_test_total)
#X_val.shape

# Selection using chi-square
#selector = SelectKBest(chi2, k=15155).fit(X_train_total, y_train_total)
#X_train_total = selector.transform(X_train_total)
#X_test_total = selector.transform(X_test_total)
#print('Training shape:')
#print(X_train_total.shape)
#print('Testing shape:')
#print(X_test_total.shape)

# Model training and fitting: Logistic Regression

In [36]:
targetencoder = LabelEncoder()
targetencoder.fit(gender_age_train.group)
y_train_total = targetencoder.transform(gender_age_train.group)
target_len = len(targetencoder.classes_)
print(target_len)

12


In [347]:
X_train,X_test,y_train,y_test = cv.train_test_split(X_train_total,y_train_total,test_size=0.005, random_state=42)

In [348]:
model = LogisticRegression(C=0.02,multi_class='multinomial',solver='newton-cg')
model.fit(X_train,y_train)

LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [349]:
y_pred = model.predict_proba(X_test)

In [350]:
log_loss(y_test,y_pred)

2.2749612582739513

In [351]:
model.score(X_test,y_test)

0.19901403922409175

In [352]:
result = model.fit(X_train_total,y_train_total)
pred = pd.DataFrame(model.predict_proba(X_test_total), index = gender_age_test.index, columns=targetencoder.classes_)
pred.head()

,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
device_id,,,,,,,,,,,,
1002079943728939269,0.001320,0.006996,0.013956,0.011760,0.033186,0.051509,0.010501,0.031108,0.069016,0.108805,0.245644,0.416199
-1547860181818787117,0.009032,0.014880,0.029417,0.061730,0.070525,0.148762,0.006344,0.118501,0.055331,0.067424,0.240890,0.177163
7374582448058474277,0.022811,0.042062,0.040158,0.136054,0.174151,0.077040,0.011671,0.024363,0.039140,0.101672,0.183772,0.147106
-6220210354783429585,0.003416,0.029785,0.009868,0.012436,0.046435,0.144354,0.065789,0.174171,0.068642,0.121052,0.169799,0.154253
-5893464122623104785,0.047394,0.065124,0.042721,0.062585,0.056626,0.043316,0.092342,0.164602,0.097607,0.102084,0.132622,0.092977


In [353]:
pred.to_csv('logreg_submission.csv',index=True)

# Model fitting :RF

In [234]:
model2 = rfc(n_estimators=300,verbose=1,n_jobs=-1)
model2.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [236]:
y_pred2 = model2.predict_proba(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.6s finished


In [237]:
log_loss(y_test,y_pred2)

3.2819344751732902

# Deep learning model

In [37]:
targetencoder = LabelEncoder().fit(gender_age_train.group)
y = targetencoder.transform(gender_age_train.group)
nclasses = len(targetencoder.classes_)
dummy_y = np_utils.to_categorical(y)

Defining functions:

In [114]:
def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator 
    #(https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0
def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    #model.add(Dense(10, input_dim=X_train_total.shape[1], init='normal', activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(150, input_dim=X_train_total.shape[1], init='normal', activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train_total.shape[1], init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [115]:
kf = KFold(len(y_train_total),n_folds = 10)
score_list=[]
val_loss_list = []
for train, test in kf:
    X_train = X_train_total[train]
    y_train = dummy_y[train]
    X_val = X_train_total[test]
    y_val = dummy_y[test]
    #print(X_val.shape)
    print('*****************************************************')
    model=baseline_model()
    fit= model.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=15,
                         samples_per_epoch=69984,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )
    scores_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), 
                                         val_samples=X_val.shape[0])
    scores = model.predict_generator(generator=batch_generatorp(X_test_total, 32, False), 
                                     val_samples=X_test_total.shape[0])
    score_list.append(scores)
    val_loss = log_loss(y_val, scores_val)
    val_loss_list.append(val_loss)
    print('logloss val {}'.format(val_loss))

*****************************************************
Epoch 1/15
62s - loss: 2.4163 - acc: 0.1398 - val_loss: 2.4392 - val_acc: 0.1253
Epoch 2/15
58s - loss: 2.3253 - acc: 0.1773 - val_loss: 2.4090 - val_acc: 0.1381
Epoch 3/15
57s - loss: 2.2856 - acc: 0.1878 - val_loss: 2.4050 - val_acc: 0.1435
Epoch 4/15
63s - loss: 2.2700 - acc: 0.1938 - val_loss: 2.3989 - val_acc: 0.1463
Epoch 5/15
61s - loss: 2.2541 - acc: 0.2020 - val_loss: 2.4011 - val_acc: 0.1478
Epoch 6/15
55s - loss: 2.2451 - acc: 0.2062 - val_loss: 2.3961 - val_acc: 0.1483
Epoch 7/15
56s - loss: 2.2342 - acc: 0.2080 - val_loss: 2.4008 - val_acc: 0.1447
Epoch 8/15
56s - loss: 2.2326 - acc: 0.2077 - val_loss: 2.4022 - val_acc: 0.1445
Epoch 9/15
56s - loss: 2.2213 - acc: 0.2148 - val_loss: 2.3995 - val_acc: 0.1451
Epoch 10/15
56s - loss: 2.2137 - acc: 0.2153 - val_loss: 2.4022 - val_acc: 0.1483
Epoch 11/15
56s - loss: 2.2085 - acc: 0.2207 - val_loss: 2.3993 - val_acc: 0.1496
Epoch 12/15
55s - loss: 2.2064 - acc: 0.2207 - val_lo

/Users/Yuyang/anaconda/lib/python3.5/site-packages/keras/engine/training.py:1403: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


In [116]:
for index,i in enumerate(val_loss_list):
    if(index==0):
        sumi = i
    else:
        sumi = i+sumi
val_loss_ave = sumi/len(val_loss_list)
print('average logloss val {}'.format(val_loss_ave))
for index,i in enumerate(score_list):
    if(index==0):
        sumi = i
    else:
        sumi = i+sumi
score_ave = sumi/len(score_list)
pred = pd.DataFrame(score_ave, index = gender_age_test.index, columns=targetencoder.classes_)
pred.to_csv('keras_v15.csv',index=True)

average logloss val 2.262167737568144


In [85]:
"""model=baseline_model()

X_train, X_val, y_train, y_val = cv.train_test_split(X_train_total, dummy_y, test_size=0.01, random_state=42)

fit= model.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=20,
                         samples_per_epoch=69984,
                         validation_data=(X_val.todense(), y_val), verbose=1
                         )

# evaluate the model
scores_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), val_samples=X_val.shape[0])
scores = model.predict_generator(generator=batch_generatorp(X_test_total, 32, False), val_samples=X_test_total.shape[0])

print('logloss val {}'.format(log_loss(y_val, scores_val)))

#Scaling to 1-0 probs
#for i in xrange(Xtest.shape[0]):
#    scores2[i,]=scores[i,]/sum(scores[i,])

pred = pd.DataFrame(scores, index = gender_age_test.index, columns=targetencoder.classes_)
#pred.head()


pred.to_csv('keras_v14.csv',index=True)
"""

Epoch 1/20
70000/69984 [==============================] - 41s - loss: 2.4466 - acc: 0.1226 - val_loss: 2.4171 - val_acc: 0.1526
Epoch 2/20
70298/69984 [==============================] - 37s - loss: 2.3851 - acc: 0.1482 - val_loss: 2.3638 - val_acc: 0.1647
Epoch 3/20
70298/69984 [==============================] - 36s - loss: 2.3409 - acc: 0.1645 - val_loss: 2.3371 - val_acc: 0.1660
Epoch 4/20
70298/69984 [==============================] - 37s - loss: 2.3196 - acc: 0.1749 - val_loss: 2.3287 - val_acc: 0.1727
Epoch 5/20
70298/69984 [==============================] - 41s - loss: 2.3017 - acc: 0.1810 - val_loss: 2.3205 - val_acc: 0.1794
Epoch 6/20
70298/69984 [==============================] - 42s - loss: 2.2919 - acc: 0.1854 - val_loss: 2.3187 - val_acc: 0.1754
Epoch 7/20
70298/69984 [==============================] - 47s - loss: 2.2829 - acc: 0.1879 - val_loss: 2.3107 - val_acc: 0.1794
Epoch 8/20
70298/69984 [==============================] - 42s - loss: 2.2769 - acc: 0.1917 - val_loss: 2

/Users/Yuyang/anaconda/lib/python3.5/site-packages/keras/engine/training.py:1403: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '
